In [1]:
BASE_DIR = "/content/drive/MyDrive/learning/8_supervision_count_bytetrack"

%cd {BASE_DIR}

/content/drive/MyDrive/Nixense_Vixion/learning/8_supervision_count_bytetrack


### Installing Packages

- ultralytics

In [2]:
!pip install ultralytics==8.0.157

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 8.5 MB/s eta 0:00:00


In [3]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.157 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/78.2 GB disk)


- ByteTrack

In [5]:
# !git clone https://github.com/ifzhang/ByteTrack.git
# %cd ByteTrack
# !pip3 install -r requirements.txt
# !python3 setup.py develop
# !pip3 install cython; pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
# !pip3 install cython_bbox
# !pip install tqdm loguru lap thop cython_bbox onemetric

- supervision

In [4]:
!pip install supervision==0.13.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 873.8 kB/s eta 0:00:00


### Imports

In [27]:
from ultralytics import YOLO
from supervision.geometry.core import Point
from supervision.utils.video import get_video_frames_generator
from supervision.utils.notebook import plot_image
from supervision.detection.core import Detections
from supervision.detection.annotate import BoxAnnotator
from supervision.draw.color import Color, ColorPalette
from supervision.utils.video import VideoSink
from supervision.utils.video import VideoInfo
from supervision.detection.line_counter import LineZone, LineZoneAnnotator


from tqdm.notebook import tqdm

### Download videos

In [10]:
%cd {BASE_DIR}
#!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-" -O vehicle-counting.mp4 && rm -rf /tmp/cookies.txt


/content/drive/MyDrive/Nixense_Vixion/learning/8_supervision_count_bytetrack


In [19]:
VIDEO_PATH_1 = f"{BASE_DIR}/vehicle-counting.mp4"


## Load YoloV8 model

In [20]:
model = YOLO("yolov8x.pt")

model.fuse()

label_map = model.model.names

YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients


In [21]:
#%cd {BASE_DIR}

!yolo task=detect mode=predict source={VIDEO_PATH_1}

Ultralytics YOLOv8.0.157 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients

video 1/1 (1/538) /content/drive/MyDrive/Nixense_Vixion/learning/8_supervision_count_bytetrack/vehicle-counting.mp4: 384x640 3 cars, 1 truck, 120.1ms
video 1/1 (2/538) /content/drive/MyDrive/Nixense_Vixion/learning/8_supervision_count_bytetrack/vehicle-counting.mp4: 384x640 4 cars, 1 truck, 29.9ms
video 1/1 (3/538) /content/drive/MyDrive/Nixense_Vixion/learning/8_supervision_count_bytetrack/vehicle-counting.mp4: 384x640 4 cars, 1 bus, 1 truck, 19.5ms
video 1/1 (4/538) /content/drive/MyDrive/Nixense_Vixion/learning/8_supervision_count_bytetrack/vehicle-counting.mp4: 384x640 4 cars, 1 bus, 1 truck, 24.0ms
video 1/1 (5/538) /content/drive/MyDrive/Nixense_Vixion/learning/8_supervision_count_bytetrack/vehicle-counting.mp4: 384x640 3 cars, 1 truck, 20.6ms
video 1/1 (6/538) /content/drive/MyDrive/Nixense_Vixion/learning/8_supervision_cou

## Initialize ByteTracker

In [22]:
from supervision.tracker.byte_tracker.core import ByteTrack

byte_tracker = ByteTrack()

### Perform Detection - Tracking - Counting

- config

In [23]:
RESULT_PATH = f"{BASE_DIR}/vehicle_counting_result.mp4"

In [24]:
x1=(3840/2)/2+270
x2=(3840/2)-100

LINE_START = Point(50, 1500)
LINE_END = Point(3840-50, 1500)

line_counter = LineZone(start=LINE_START,end=LINE_END)
line_annotator = LineZoneAnnotator(thickness=4, text_thickness=4, text_scale=2)

- detection - tracking - counting

In [44]:
generator = get_video_frames_generator(VIDEO_PATH_1)

box_annotator = BoxAnnotator(color=ColorPalette.default(),thickness=4, text_thickness=4,text_scale=2)

#iterator = iter(generator)

#frame = next(iterator)

video_info = VideoInfo.from_video_path(VIDEO_PATH_1)
with VideoSink(RESULT_PATH, video_info) as sink:
  for frame in tqdm(generator,total=video_info.total_frames):

    results = model(frame)[0]
    detections = Detections.from_ultralytics(results)

    detections = byte_tracker.update_with_detections(detections=detections)

    # detections = Detections(
    #     xyxy=results.boxes.xyxy.cpu().numpy(),
    #     confidence=results.boxes.conf.cpu().numpy(),
    #     class_id=results.boxes.cls.cpu().numpy().astype(int)
    # )
    # tracks = byte_tracker.update(
    #     output_results=detections2boxes(detections=detections),
    #     img_info=frame.shape,
    #     img_size=frame.shape
    # )
    # tracker_id = match_detections_with_tracks(detections=detections,tracks=tracks)
    # detections.tracker_id=np.array(tracker_id)


    labels=[f"{label_map[class_id]} {confidence:0.2f} -track_id:{tracker_id}" for _,_,confidence,class_id,tracker_id in detections]

    line_counter.trigger(detections=detections)

    annotated_frame = box_annotator.annotate(scene=frame, detections=detections,labels=labels)
    line_annotator.annotate(frame=frame,line_counter=line_counter)

    sink.write_frame(frame)
    #show_frame_in_notebook(annotated_frame,(10,10))

  0%|          | 0/538 [00:00<?, ?it/s]


0: 384x640 3 cars, 1 truck, 63.3ms
Speed: 4.4ms preprocess, 63.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 64.4ms
Speed: 4.8ms preprocess, 64.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 2 trucks, 63.2ms
Speed: 11.1ms preprocess, 63.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 truck, 69.9ms
Speed: 3.4ms preprocess, 69.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 63.5ms
Speed: 4.0ms preprocess, 63.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 63.4ms
Speed: 3.2ms preprocess, 63.4ms inference, 3.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 64.2ms
Speed: 6.4ms preprocess, 64.2ms inference, 10.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 truck, 69.0ms
Speed: 3.3ms preprocess, 69.0m